In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVR
from scipy.stats import uniform, randint

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [36]:
df = pd.read_csv('Данные для построения модели, задача - Регрессия для IC50.csv')
df

,EState_VSA4,Chi2n,Chi0v,Chi2v,SlogP_VSA5,Chi0n,Chi1,SlogP_VSA2,SI,SlogP_VSA6,...,LabuteASA,BCUT2D_CHGHI,MaxAbsEStateIndex,EState_VSA2,Chi0,BCUT2D_MWLOW,BCUT2D_MRLOW,MinAbsPartialCharge,"CC50, mM",Target Feature
0,24.925325,12.058078,19.534409,12.058078,105.750639,19.534409,13.127794,24.512883,28.125000,9.984809,...,173.630124,2.600532,5.094096,0.000000,20.208896,9.700470,0.193481,0.038844,175.482382,0.795141
1,23.919494,12.648545,19.794682,12.648545,105.750639,19.794682,13.127794,25.173194,7.000000,0.000000,...,174.939204,2.614066,3.961417,0.000000,20.208896,9.689226,0.120322,0.012887,5.402819,-0.112478
2,23.919494,15.671216,23.689110,15.671216,105.750639,23.689110,14.595754,62.329944,0.720000,0.000000,...,201.238858,2.665274,2.627117,0.000000,23.794682,9.681293,-0.922902,0.094802,161.142320,2.349877
3,24.925325,12.411631,20.241516,12.411631,112.171461,20.241516,13.627794,24.512883,63.235294,9.984809,...,179.995066,2.600529,5.097360,0.000000,20.916003,9.700497,0.193510,0.038844,107.855654,0.231883
4,29.631406,13.613700,22.617677,13.613700,91.194256,22.617677,16.526773,11.423370,1.300000,58.515746,...,211.919602,2.602486,5.150510,10.829981,24.639617,9.700386,0.193687,0.062897,139.270991,2.029917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,38.524930,11.409461,18.825334,11.409461,66.219879,18.825334,14.097861,25.017713,1.129017,11.649125,...,178.490760,2.726237,12.934891,52.409521,21.637464,9.344314,-0.196385,0.317890,34.999650,1.491363
997,38.524930,12.636189,21.810933,12.636189,73.143616,21.810933,16.402391,36.966734,1.062484,11.649125,...,207.296970,2.725543,13.635345,52.347395,25.499271,9.343622,-0.187625,0.327562,33.999415,1.505149
998,44.277783,13.248561,24.449891,14.234160,72.640700,23.633394,17.940396,48.975357,1.096761,11.649125,...,230.149965,2.725818,13.991690,46.932405,27.620591,9.343613,-0.187687,0.327887,33.999458,1.491360
999,6.420822,11.893254,23.380977,11.893254,66.219879,23.380977,17.309003,58.099656,1.031272,0.000000,...,218.836986,2.730109,13.830180,30.512100,27.921921,9.364015,-0.225309,0.312509,32.999644,1.505136


In [37]:
X, y = df.drop('Target Feature', axis=1), df['Target Feature']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Target Feature', axis=1), df['Target Feature'], test_size=0.3, random_state=42)

#### Контрольной метрика для задачи регрессии будем считать R^2 score. Она будет записана в атрибут assessment_criterion BestRegressionModel класса

In [39]:
class BestRegressionModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели регресии
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'R^2',
            'value': 0.5
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LinearRegression(),
                'best_model': None,
                'best_params': None,
                'params': {},
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestRegressor(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': ElasticNet(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'alpha': [0.01, 0.1, 1, 10],
                    'l1_ratio': [0.2, 0.5, 0.8]
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVR(),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            self.models[model]['metrics']['MSE'] = mean_squared_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['MAE'] = mean_absolute_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['R^2'] = r2_score(y, self.models[model]['data']['y_pred'])
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"MSE: {self.models[model]['metrics']['MSE']:.4f}")
            print(f"MAE: {self.models[model]['metrics']['MAE']:.4f}")
            print(f"R^2: {self.models[model]['metrics']['R^2']:.4f}")
            print()
            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return 10 ** result

#### Подбор гипперпараметров для моделей

In [40]:
best_regression_model = BestRegressionModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LinearRegression() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели LinearRegression(): {}

Подбор гипперпараметров для модели RandomForestRegressor(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestRegressor(random_state=42): {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 124}

Подбор гипперпараметров для модели ElasticNet(random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели ElasticNet(random_state=42): {'alpha': 0.01, 'l1_ratio': 0.2}

Подбор гипперпараметров для модели SVR() методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию параме

#### Поиск лучшей модели

In [41]:
best_regression_model.get_best_model()

Обучаем модель LinearRegression() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LinearRegression():
MSE: 0.5477
MAE: 0.5971
R^2: 0.3810

Обучаем модель RandomForestRegressor(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestRegressor(random_state=42):
MSE: 0.0270
MAE: 0.0699
R^2: 0.9694

Обучаем модель ElasticNet(random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели ElasticNet(random_state=42):
MSE: 0.5246
MAE: 0.5848
R^2: 0.4072

Обучаем модель SVR() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандарт

#### Формирование предсказания лучшей моделью

In [42]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели RandomForestRegressor(max_depth=10, min_samples_leaf=2, min_samples_split=4,
                      n_estimators=124, random_state=42)
Предсказание сформировано


array([1.06559587e+03, 2.84483248e+01, 8.10033751e+01, 2.96053679e+01,
       2.62531239e+01, 1.79299541e+02, 3.69172261e+02, 9.57656101e+02,
       7.15874878e-01, 7.48651371e+01, 4.17389197e+01, 6.24197013e+00,
       7.10073557e+00, 1.03256057e+01, 2.91013647e+01, 6.34559082e+00,
       3.14620051e+02, 3.46228098e-02, 5.53151294e+02, 1.23267562e+01,
       7.72844147e+01, 3.10772467e+01, 2.05254132e+01, 9.78702986e+02,
       2.85041219e+02, 1.01110485e+01, 1.71249200e+03, 4.15911930e+00,
       5.29569577e+00, 1.64481748e-01, 1.29858880e+01, 2.70340104e+01,
       9.31168832e+02, 4.48380031e+02, 1.29877762e+03, 3.47278124e+01,
       8.43063599e+02, 1.75637512e+01, 3.01390843e+01, 7.99388798e+00,
       1.25706984e+01, 2.42030585e+01, 6.32844860e+01, 9.98446408e+01,
       8.54021425e+02, 9.00726654e+01, 2.08843925e+02, 2.20666414e+01,
       1.93577810e+02, 1.02771173e+02, 4.34203077e+00, 8.68056101e-02,
       3.25330420e+02, 2.11889599e+01, 3.90262255e+02, 1.37862021e+03,
      